In [39]:
#imports
from __future__ import division

import xarray as xr
import numpy as np
from itertools import product
from wrf import getvar, get_basemap
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from matplotlib import path
import geopandas as gpd
import shapely
#import pandas as pd

In [40]:
#import basemap proj from wrf
file1_wrf_d01 = '/glade/scratch/molina/WRF_HYSPLIT_proj/wrf4km_ens_1/wrfout_d01_2017-01-15_09:00:00'
ncfile1_wrf_d01 = Dataset(file1_wrf_d01)
while True:
    p = np.array([0])
    if len(p) == 1:
        try: 
            p = getvar(ncfile1_wrf_d01, "pressure")
        except ValueError:
            continue
    if len(p) != 1:
        print('First Opened...')
        break
bm = get_basemap(p)

First Opened...


In [41]:
#skin temp import
data_skin = xr.open_dataset('/glade/work/molina/DATA/jan2017_synoptic/narr_skin_casestudy.nc', decode_cf=True)
data_skin_temp = data_skin.sfc_temp.values
data_skin_lats = data_skin.lat.values
data_skin_lons = data_skin.lon.values

In [42]:
#create regions

#MU region
box_lats = [ 18.0, 25.0, 25.0, 18.0]
box_lons = [-88.,-88.,-85.,-84.0]
box_x, box_y = box_lons,box_lats
box_xy = zip(box_x, box_y)              
box_path = path.Path(np.array(list(box_xy)))
box_lats2 = [ 18.0, 21.0, 21.0, 18.0]
box_lons2 = [-85.0,-85.0,-77.0,-77.0]
box_x2, box_y2 = box_lons2,box_lats2
box_xy2 = zip(box_x2, box_y2)              
box_path2 = path.Path(np.array(list(box_xy2)))

#gom plume region 
box_lats3 = [ 25.0, 30.5, 30.5, 25.0]
box_lons3 = [-84.0,-90.0,-83.0,-81.0]
box_x3, box_y3 = box_lons3, box_lats3
box_xy3 = zip(box_x3, box_y3)              
box_path3 = path.Path(np.array(list(box_xy3)))

#gom basin region
can = gpd.GeoDataFrame.from_file("/glade/work/molina/DATA/ocean_shapefiles/World_Seas_WGS84.shp")
gom = can[can['NAME']=='Gulf of Mexico']
studyarea = shapely.geometry.box(bm.llcrnrlon, bm.llcrnrlat, 
                                 bm.urcrnrlon, bm.urcrnrlat)
selection = gom[gom.geometry.intersects(studyarea)]
for poly in selection.geometry:
    if poly.geom_type == 'Polygon':
        mpoly = shapely.ops.transform(bm, poly)
    elif poly.geom_type == 'MultiPolygon':
        for subpoly in poly:
            mpoly = shapely.ops.transform(bm, poly)
    else:
        print(poly, ' is neither a polygon nor a multi-polygon. Skipping it.')

In [43]:
#arrays to create from regions
file_of_skintemp_1 = np.zeros((data_skin_temp.shape))
file_of_scatters_1 = np.zeros((data_skin_lats.shape))
file_of_skintemp_2 = np.zeros((data_skin_temp.shape))
file_of_scatters_2 = np.zeros((data_skin_lats.shape))
file_of_skintemp_3 = np.zeros((data_skin_temp.shape))
file_of_scatters_3 = np.zeros((data_skin_lats.shape))

In [47]:
#looping through regions and data to filter into regions of interest
for numer, (i, j) in enumerate(product(range(data_skin_lats[:,0].shape[0]),range(data_skin_lats[0,:].shape[0]))):    
    
    if numer%1000 == 0:
        print('%.2f' % ((numer/(data_skin_lats.shape[0]*data_skin_lats.shape[1]))*100)+'% complete...')
    
    #checking first MU region
    if box_path.contains_point([data_skin_lons[i,j],data_skin_lats[i,j]]):
        xtemp, ytemp = bm(data_skin_lons[i,j],data_skin_lats[i,j])
        #checking if over land
        if not bm.is_land(xtemp, ytemp):
            file_of_skintemp_1[:,i,j] = data_skin_temp[:,i,j]
            file_of_scatters_1[i,j] = 1.
    #only bother doing this if a grid point hasn't been changed from zero
    if np.all(file_of_skintemp_1[:,i,j]) == 0:
        #checking second MU region
        if box_path2.contains_point([data_skin_lons[i,j],data_skin_lats[i,j]]):
            xtemp, ytemp = bm(data_skin_lons[i,j],data_skin_lats[i,j])
            #checking if over land
            if not bm.is_land(xtemp, ytemp):
                file_of_skintemp_1[:,i,j] = data_skin_temp[:,i,j]
                file_of_scatters_1[i,j] = 1.
    #checking GoM plume region
    if box_path3.contains_point([data_skin_lons[i,j],data_skin_lats[i,j]]):
        xtemp, ytemp = bm(data_skin_lons[i,j],data_skin_lats[i,j])
        #checking if over land
        if not bm.is_land(xtemp, ytemp):
            file_of_skintemp_2[:,i,j] = data_skin_temp[:,i,j]
            file_of_scatters_2[i,j] = 1.
    #checking if grid point is in GoM basin
    if mpoly.contains(shapely.geometry.Point(bm(data_skin_lons[i,j],data_skin_lats[i,j]))):
        xtemp, ytemp = bm(data_skin_lons[i,j],data_skin_lats[i,j])
        #checking if point is over land
        if not bm.is_land(xtemp, ytemp):
            file_of_skintemp_3[:,i,j] = data_skin_temp[:,i,j]
            file_of_scatters_3[i,j] = 1.

0.00% complete...
1.03% complete...
2.07% complete...
3.10% complete...
4.14% complete...
5.17% complete...
6.21% complete...
7.24% complete...
8.28% complete...
9.31% complete...
10.34% complete...
11.38% complete...
12.41% complete...
13.45% complete...
14.48% complete...


KeyboardInterrupt: 

In [ ]:
#create lat lon coords for data set
x, y = bm(data_skin_lons, data_skin_lats)

In [ ]:
final_filed = xr.Dataset({'mu_region':(['time','x','y'],file_of_skintemp_1),
                          'gom_plume':(['time','x','y'],file_of_skintemp_2),
                          'gom_basin':(['time','x','y'],file_of_skintemp_3),
                          'mu_region_scat':(['x','y'],file_of_scatters_1),
                          'gom_plume_scat':(['x','y'],file_of_scatters_2),
                          'gom_basin_scat':(['x','y'],file_of_scatters_3)},
                          coords={'time':(['time'],data_skin.coords['time']),
                                  'lon':(['x','y'],x),
                                  'lat':(['x','y'],y)},
                          attrs={'File Author':'Maria J. Molina'})
        

final_filed.to_netcdf('perturb_skin_ssts.nc')

In [55]:
import pandas as pd
pd.to_datetime(data_skin.coords['time'].values)

DatetimeIndex(['2017-01-15 09:00:00', '2017-01-15 12:00:00',
               '2017-01-15 15:00:00', '2017-01-15 18:00:00',
               '2017-01-15 21:00:00', '2017-01-16 00:00:00',
               '2017-01-16 03:00:00', '2017-01-16 06:00:00',
               '2017-01-16 09:00:00', '2017-01-16 12:00:00',
               '2017-01-16 15:00:00', '2017-01-16 18:00:00',
               '2017-01-16 21:00:00', '2017-01-17 00:00:00',
               '2017-01-17 03:00:00', '2017-01-17 06:00:00',
               '2017-01-17 09:00:00', '2017-01-17 12:00:00',
               '2017-01-17 15:00:00', '2017-01-17 18:00:00',
               '2017-01-17 21:00:00', '2017-01-18 00:00:00',
               '2017-01-18 03:00:00', '2017-01-18 06:00:00',
               '2017-01-18 09:00:00', '2017-01-18 12:00:00',
               '2017-01-18 15:00:00', '2017-01-18 18:00:00',
               '2017-01-18 21:00:00', '2017-01-19 00:00:00',
               '2017-01-19 03:00:00', '2017-01-19 06:00:00',
               '2017-01-